# Time series predictions: PyTorch implementation

References and futher readings:
1. [LSTMs for Time Series in PyTorch](https://www.jessicayung.com/lstms-for-time-series-in-pytorch/) by Jessica Yung
2. [Time Series Prediction using LSTM with PyTorch in Python](https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/) by Usman Malik

## Libraries import

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import colorlover as cl
init_notebook_mode(connected=True)